# 앙상블(Ensemble)
여러 개의 약한 학습기(모델)를 결합하여 하나의 강력한 예측기를 만드는 방법
－ 백지장도 맞들면 낫다
－ Bagging、 Boosting

# 랜덤 포레스트(Random Forest)
**여러 개의 결정트리(Decision Tree)** 를 **무작위**로 만들어 **평균**을 내는 앙상블 기법
- 여러 개의 트리를 학습시켜 각각의 예측 결과를 평균 or 다수결로 결정
- 각 트리는 학습할 때 **데이터와 변수(특징) 무작위 선택**
- 서로 다른 관점을 가진 트리들 -> **과적합(Overfitting) 방지**와 일반화 성능 향상
- *변수 중요도* 확인 가능

##### 진행 과정
1. 학습 데이터에서 무작위 샘플(Bootstrap)을 뽑음
2. 각 샘플로 결정트리 학습
3. 새로운 데이터가 들어오면 여러 트리의 결과를 **평균(회귀)** 또는 **다수결(분류)**
4. 최종 결과 결정

---

> ##### 부트스트랩(Bootstrap)
- 데이터를 **무작위**로 **복원추출**해서 여러 개의 학습 데이터를 만드는 방법
- 각 dataset은 원본과 크기는 같지만, *표본 구성이 조금씩 다름*

<br>

> ##### Bagging (Bootstrap Aggregating)
1. Bootstrap 샘플링으로 여러 개의 데이터셋을 만듦
2. 각 데이터셋으로 서로 다른 모델(보통 같은 알고리즘) 학습
3. 예측 시, 모든 모델의 결과를 평균 or 투표로 종합 => 모델 간 편향(Bias) 감소, 분산(Variance) 안정화
- 대표 알고리즘: 랜덤 포레스트

<br>

> ##### Boosting
- 잘못 예측한 데이터에 점점 더 집중하면서, 모델을 순차적으로 학습시키는 방법
- 한 번에 여러 모델을 독립적으로 학습시키는 Bagging과 달리, **Boosting은 순차적(Sequential) 으로 모델 학습**
- 이전 모델이 틀린 데이터를 다음 모델이 더 잘 맞추도록 **가중치(weight) 조정** → 즉, “이전 모델의 실수를 보완하는” 학습 방법
－ 대표 알고리즘： AdaBoost (Adaptive Boosting), Gradient Boosting, XGBoost / LightGBM / CatBoost
1. 첫 번째 모델 학습
2. 예측이 틀린 데이터에 가중치를 높임
3. 두 번째 모델이 이 데이터를 더 중요하게 학습
4. 여러 모델을 순서대로 연결해 최종 예측을 결합

<br>


| 구분            | Bootstrap   | Bagging              | Boosting            |
| ------------- | ----------- | -------------------- | ------------------- |
| **핵심 개념**     | 복원추출로 표본 생성 | 여러 모델을 병렬 학습 후 평균/투표 | 모델을 순차적으로 연결해 오차 보완 |
| **학습 방식**     | 데이터 샘플링     | 병렬(Parallel)         | 순차(Sequential)      |
| **목적**        | 데이터 다양성 확보  | 분산 감소(안정성↑)          | 편향 감소(정확도↑)         |
| **대표 알고리즘**   | -           | Random Forest        | AdaBoost, XGBoost 등 |
| **과적합 방지 효과** | 간접적         | 큼                    | 조심 필요(파라미터 조정 중요)   |


In [1]:
# 🌲 Random Forest 실습
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# 1️⃣ 데이터 불러오기
iris = load_iris()
X = iris.data
y = iris.target

# 2️⃣ 학습용/테스트용 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [2]:
# 3️⃣ 랜덤 포레스트 모델 생성
rf = RandomForestClassifier(
    n_estimators=100,      # 트리 개수
    max_depth=None,        # 트리 최대 깊이 (None이면 자동)
    random_state=42
)

# 4️⃣ 모델 학습
rf.fit(X_train, y_train)

# 5️⃣ 예측
y_pred = rf.predict(X_test)

In [3]:
# 6️⃣ 평가
print("정확도:", accuracy_score(y_test, y_pred))
print("\n분류 리포트:\n", classification_report(y_test, y_pred))

정확도: 1.0

분류 리포트:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [4]:
# 7️⃣ 변수 중요도 확인
import pandas as pd
feature_importance = pd.DataFrame({
    'Feature': iris.feature_names,
    'Importance': rf.feature_importances_
}).sort_values('Importance', ascending=False)
print("\n📊 변수 중요도:\n", feature_importance)


📊 변수 중요도:
              Feature  Importance
2  petal length (cm)    0.439994
3   petal width (cm)    0.421522
0  sepal length (cm)    0.108098
1   sepal width (cm)    0.030387


`n_estimators`: 트리 개수가 많을수록 일반적으로 성능이 좋아지지만 계산량 증가

`max_depth`: 트리의 복잡도 제한 (과적합 방지)

`feature_importances_`: 어떤 특징이 분류에 더 중요한지 알 수 있음